<a href="https://colab.research.google.com/github/NeuromatchAcademy/course-content-dl/blob/w2d1_fixes/tutorials/W2D1_ConvnetsAndRecurrentNeuralNetworks/W2D1_Tutorial2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 2: Introduction to RNNs

**Week 2, Day 1: Convnets And Recurrent Neural Networks**

**By Neuromatch Academy**

__Content creators:__ Dawn McKnight, Richard Gerum, Cassidy Pirlot, Rohan Saha, Liam Peet-Pare, Saeed Najafi, Alona Fyshe

__Content reviewers:__ Saeed Salehi, Lily Cheng, Yu-Fang Yang, Polina Turishcheva, Nina Kudryashova, Kelson Shilling-Scrivo

__Content editors:__ Gagana B, Nina Kudryashova

__Production editors:__ Anmol Gupta, Spiros Chavlis 

__Post-Production team:__ Gagana B, Spiros Chavlis

*Based on material from:* Konrad Kording, Hmrishav Bandyopadhyay, Rahul Shekhar, Tejas Srivastava

**Our 2021 Sponsors, including Presenting Sponsor Facebook Reality Labs**

<p align='center'><img src='https://github.com/NeuromatchAcademy/widgets/blob/master/sponsors.png?raw=True'/></p>

---
# Tutorial Objectives
At the end of this tutorial, we will be able to:
- Understand the structure of a Recurrent Neural Network (RNN)
- Build a simple RNN model

 

In [1]:
# @title Tutorial slides

from IPython.display import IFrame
IFrame(src=f"https://mfr.ca-1.osf.io/render?url=https://osf.io/5asx2/?direct%26mode=render%26action=download%26mode=render", width=854, height=480)

These are the slides for the videos in this tutorial. If you want to download locally the slides, click [here](https://osf.io/5asx2/download).


---
# Setup

In [2]:
# @title Install dependencies
!pip install livelossplot --quiet
!pip install unidecode --quiet

!pip install git+https://github.com/NeuromatchAcademy/evaltools --quiet
from evaltools.airtable import AirtableForm

# Generate airtable form
atform = AirtableForm('appn7VdPRseSoMXEG','W2D1_T2','https://portal.neuromatchacademy.org/api/redirect/to/351ca652-13d8-4e31-be28-30153d03e639')

     |████████████████████████████████| 235 kB 4.1 MB/s 


In [3]:
# Imports
import time
import math
import torch
import string
import random
import unidecode
import numpy as np
import matplotlib.pyplot as plt

import torch.nn as nn

from tqdm.notebook import tqdm

In [4]:
# @title Figure settings
import ipywidgets as widgets       # Interactive display
%config InlineBackend.figure_format = 'retina'
plt.style.use("https://raw.githubusercontent.com/NeuromatchAcademy/content-creation/main/nma.mplstyle")

plt.rcParams["mpl_toolkits.legacy_colorbar"] = False

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

In [5]:
# @title Helper functions
# https://github.com/spro/char-rnn.pytorch

def read_file(filename):
  """
  Helper function to read file 

  Args: 
    filename: string
      FileName

  Returns: 
    file: string
      Contents of file
    And file length
  """
  file = unidecode.unidecode(open(filename).read())
  return file, len(file)


def char_tensor(string):
  """
  Turning a string into a tensor

  Args:
    string: string
      Input string
    
  Returns:
    tensor: torch.tensor
      Tensor from input string
  """
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
    try:
      tensor[c] = all_characters.index(string[c])
    except:
      continue
  return tensor


def time_since(since):
  """
  Readable time elapsed

  Args: 
    since: time
      Input time

  Returns:
    out: string
      Time elapsed since since. 
  """
  s = time.time() - since
  m = math.floor(s / 60)
  s -= m * 60
  out = f"{m}min {s}sec"
  return out


def generate(decoder, prime_str='A', predict_len=100, 
             temperature=0.8,
             device='cpu'):
  """
  Function to generate predicted future sequence

  Args: 
    decoder: nn.module
      Model
    prime_str: string
      Prime string [default: A]
    predict_len: int
      Predict length [default: 100]
    temperature: float
      Temperature [default: 0.8]
    device: string
      GPU/CUDA if available, CPU otherwise

  Returns:
    predicted: string
      Predicted sequence
  """
  hidden = decoder.init_hidden(1)
  prime_input = char_tensor(prime_str).unsqueeze(0)

  hidden = hidden.to(device)
  prime_input = prime_input.to(device)
  predicted = prime_str

  # Use priming string to "build up" hidden state
  for p in range(len(prime_str) - 1):
    _, hidden = decoder(prime_input[:,p], hidden)

  inp = prime_input[:,-1]

  for p in range(predict_len):
    output, hidden = decoder(inp, hidden)

    # Sample from the network as a multinomial distribution
    output_dist = output.data.view(-1).div(temperature).exp()
    top_i = torch.multinomial(output_dist, 1)[0]

    # Add predicted character to string and use as next input
    predicted_char = all_characters[top_i]
    predicted += predicted_char
    inp = char_tensor(predicted_char).unsqueeze(0)
    inp = inp.to(device)

  return predicted

In [6]:
# @title Set random seed

# @markdown Executing `set_seed(seed=seed)` you are setting the seed

# For DL its critical to set the random seed so that students can have a
# baseline to compare their results to expected results.
# Read more here: https://pytorch.org/docs/stable/notes/randomness.html

# Call `set_seed` function in the exercises to ensure reproducibility.
import random
import torch

def set_seed(seed=None, seed_torch=True):
  """
  Function that controls randomness. 
  NumPy and random modules must be imported.

  Args:
    seed : Integer
      A non-negative integer that defines the random state. Default is `None`.
    seed_torch : Boolean
      If `True` sets the random seed for pytorch tensors, so pytorch module
      must be imported. Default is `True`.

  Returns:
    Nothing.
  """
  if seed is None:
    seed = np.random.choice(2 ** 32)
  random.seed(seed)
  np.random.seed(seed)
  if seed_torch:
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

  print(f'Random seed {seed} has been set.')


# In case that `DataLoader` is used
def seed_worker(worker_id):
  """
  DataLoader will reseed workers following randomness in
  multi-process data loading algorithm.

  Args:
    worker_id: integer
      ID of subprocess to seed. 0 means that
      the data will be loaded in the main process
      Refer: https://pytorch.org/docs/stable/data.html#data-loading-randomness for more details

  Returns:
    Nothing
  """
  worker_seed = torch.initial_seed() % 2**32
  np.random.seed(worker_seed)
  random.seed(worker_seed)

In [7]:
#@title Set device (GPU or CPU). Execute `set_device()`
# especially if torch modules used.

# Inform the user if the notebook uses GPU or CPU.

def set_device():
  """
  Set the device. CUDA if available, CPU otherwise

  Args:
    None

  Returns:
    Nothing
  """
  device = "cuda" if torch.cuda.is_available() else "cpu"
  if device != "cuda":
    print("WARNING: For this notebook to perform best, "
        "if possible, in the menu under `Runtime` -> "
        "`Change runtime type.`  select `GPU` ")
  else:
    print("GPU is enabled in this notebook.")

  return device

In [8]:
SEED = 2021
set_seed(seed=SEED)
DEVICE = set_device()

Random seed 2021 has been set.


---
# Section 1: Recurrent Neural Networks (RNNs)

*Time estimate: ~20mins*

In [9]:
# @title Video 1: RNNs
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = "https://player.bilibili.com/player.html?bvid={0}&page={1}".format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id=f"BV1L44y1m7PP", width=854, height=480, fs=1)
  print("Video available at https://www.bilibili.com/video/{0}".format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id=f"PsZjS125lLs", width=854, height=480, fs=1, rel=0)
  print("Video available at https://youtube.com/watch?v=" + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

# Add event to airtable
atform.add_event('Video 1: RNNs')

display(out)

RNNs are compact models that operate over timeseries, and have the ability to remember past input. They also save parameters by using the same weights at every time step.  If you've heard of Transformers, those models don't have this kind of temporal weight sharing, and so they are *much* larger.

The code below is adapted from [this github repository](https://github.com/spro/char-rnn.pytorch).

In [10]:
# https://github.com/spro/char-rnn.pytorch

class CharRNN(nn.Module):
  """
  Recurrent Neural Network Implementation
  """

  def __init__(self, input_size, hidden_size, output_size,
               model="gru", n_layers=1):
    """
    Initialise CharRNN parameters

    Args:
      input_size: int
        Size of the input layer.
      hidden_size: int
        Size of the hidden layers.
      output_size: int
        Size of the output layer.
      model: string
        `model` can take the values "gru", "rnn", "lstm". Default is "gru".
      n_layers: int
        Number of layers [default: 1]
    
    Returns: 
      Nothing
    """
    super(CharRNN, self).__init__()
    self.model = model.lower()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    self.encoder = nn.Embedding(input_size, hidden_size)
    if self.model == "gru":
      self.rnn = nn.GRU(hidden_size, hidden_size, n_layers)
    elif self.model == "lstm":
      self.rnn = nn.LSTM(hidden_size, hidden_size, n_layers)
    elif self.model == "rnn":
      self.rnn = nn.RNN(hidden_size, hidden_size, n_layers)
    self.decoder = nn.Linear(hidden_size, output_size)

  def forward(self, input, hidden):
    """
    Forward pass of CharRNN

    Args:
      input: torch.tensor
        Input to CharRNN
      hidden: int
        Dimension of hidden layer

    Returns:
      output: torch.tensor
        Output of CharRNN
      hidden: torch.tensor
        Output of CharRNN hidden layer
    """
    batch_size = input.size(0)
    encoded = self.encoder(input)
    output, hidden = self.rnn(encoded.reshape(1, batch_size, -1), hidden)
    output = self.decoder(output.reshape(batch_size, -1))
    return output, hidden

  def init_hidden(self, batch_size):
    """
    Initialise hidden dimension of CharRNN

    Args: 
      batch_size: int
        Batch Size
    
    Returns: 
      torch.zeros(self.n_layers, batch_size, self.hidden_size), torch.zeros(self.n_layers, batch_size, self.hidden_size) if LSTM
      torch.zeros(self.n_layers, batch_size, self.hidden_size) otherwise
    """
    if self.model == "lstm":
      return (torch.zeros(self.n_layers, batch_size, self.hidden_size), torch.zeros(self.n_layers, batch_size, self.hidden_size))

    return torch.zeros(self.n_layers, batch_size, self.hidden_size)

This next section of code takes care of training the RNN on several of Mark Twain's books. In this short section, we won't dive into the code, but you'll get to learn a lot more about RNNs in a few days! For now, we are just going to observe the training process.

In [11]:
# @title Run Me to get the data
import requests

url = 'https://raw.githubusercontent.com/NeuromatchAcademy/course-content-dl/main/tutorials/W2D1_ConvnetsAndRecurrentNeuralNetworks/static/twain.txt'
r = requests.get(url, stream=True)

with open('twain.txt', 'wb') as fd:
  fd.write(r.content)

One cool thing about RNNs is that they can be used to _generate_ language based on what the network sees during training. As the network makes predictions, instead of confirming of those predictions are correct against some training text, we just feed them back into the model as the next observed token.  Starting from a random vector for the hidden state, we can generate many original sentences! And what the network generates will reflect the text it was trained on.

In [12]:
# https://github.com/spro/char-rnn.pytorch

def random_training_set(file, file_len, chunk_len, batch_size,
                        device='cpu', seed=0):
  """
  Generates random training set

  Args: 
    file: string
      File
    file_len: int
      Length of file
    chunk_len: int
      Length of chunk
    batch_size: int
      Batch size
    device: string
      GPU/CUDA if available. CPU otherwise [default: CPU]
    seed: int
      Set seed for reproducibility [default: 0]

  Returns:
    inp: torch.tensor
      Input tensor
    target: torch.tensor
      Targets
    chunk_len: int
      Length of chunk
    batch_size: int
      Batch size
    device: string
      GPU/CUDA if available. CPU otherwise [default: CPU]
  """
  random.seed(seed)

  inp = torch.LongTensor(batch_size, chunk_len).to(device)
  target = torch.LongTensor(batch_size, chunk_len).to(device)

  for bi in range(batch_size):
    start_index = random.randint(0, file_len - chunk_len - 1)
    end_index = start_index + chunk_len + 1
    chunk = file[start_index:end_index]
    inp[bi] = char_tensor(chunk[:-1])
    target[bi] = char_tensor(chunk[1:])

  return inp, target, chunk_len, batch_size, device


def train(decoder, criterion, inp, target, chunk_len, batch_size, device):
  """
  Training function

  Args:
    decoder: nn.module
      Decoder model
    criterion: function
      Loss function
    inp: torch.tensor
      Input
    target: torch.tensor
      Targets
    chunk_len: int
      Length of chunk
    batch_size: int
      Batch size
    device: string
      GPU/CUDA if available. CPU otherwise [default: CPU]
    

  Returns:
    Decoder loss
  """
  hidden = decoder.init_hidden(batch_size)
  decoder.zero_grad()
  loss = 0

  for c in range(chunk_len):
    output, hidden = decoder(inp[:, c].to(device), hidden.to(device))
    loss += criterion(output.reshape(batch_size, -1), target[:,c])

  loss.backward()
  decoder_optimizer.step()
  return loss.item() / chunk_len

First, let's load the text file, and define the model and its hyperparameters.

In [13]:
# Reading and un-unicode-encoding data
all_characters = string.printable
n_characters = len(all_characters)

# Load the text file
file, file_len = read_file('twain.txt')

# Hyperparams
batch_size = 50
chunk_len = 200
model = "rnn"  # Other options: `lstm`, `gru`

n_layers = 2
hidden_size = 200
learning_rate = 0.01

# Define the model, optimizer, and the loss criterion
decoder = CharRNN(n_characters, hidden_size, n_characters,
                  model=model, n_layers=n_layers)
decoder.to(DEVICE)

decoder_optimizer = torch.optim.Adagrad(decoder.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

Let's try it! Run the code below. As the network trains, it will output samples of generated text every 25 epochs. Notice that as the training progresses, the model learns to spell short words, then learns to string some words together, and eventually can produce meaningful sentences (sometimes)! Keep in mind that this is a relatively small network, and doesn't employ some of the cool things you'll learn about later in the week (e.g., LSTMs, though you can change that in the code below by changing the value of the `model` variable if you wish!)

After running the model, and observing the output, get together with your pod, and talk about what you noticed during training. Did your network produce anything interesting? Did it produce anything characteristic of Twain?  

**Note:** training for the full 2000 epochs is likely to take a while, so you may need to stop it before it finishes. If you have time left, set `n_epochs` to 2000 below.

In [14]:
n_epochs = 1000   # Initial was set to 2000

print_every = 50  # Frequency of printing the outputs

start = time.time()
all_losses = []
loss_avg = 0

print(f"Training for {n_epochs} epochs...\n")
for epoch in tqdm(range(1, n_epochs + 1), position=0, leave=True):
  loss = train(decoder, criterion,
               *random_training_set(file, file_len, chunk_len, batch_size,
                                    device=DEVICE, seed=epoch))
  loss_avg += loss

  if epoch % print_every == 0:
    print(f"[{time_since(start)} {epoch/n_epochs * 100}%) {loss:.4f}]")
    print(f"{generate(decoder, prime_str='Wh', predict_len=150, device=DEVICE)}")

Training for 1000 epochs...



  0%|          | 0/1000 [00:00<?, ?it/s]

in 20.092137336730957sec 5.0%) 2.1638]
Whyd, Thon.

of to dekt seed sortectisthen stuicecty beall ilead, of onsingon it the dile mint hean whend witure to speaghan the say and allather a them
in 39.96975564956665sec 10.0%) 1.9462]
Wher saing forted shat his, and see instent st@on cother. And by pook a plant. and to beding.  Ildn't shen inde doch the bupcty, wouks it was the cound 
in 59.83513069152832sec 15.0%) 1.9125]
Whan and the was prepricely
firste abath to thinded the start the noses, and mace praked wesandy goied, sone to the his sitere had gaspain:

"She hunger
in 19.724196910858154sec 20.0%) 1.8870]
Where he fure filled to seefry, and says pont to king, over, in a do lece this dome to apped of so the lious to-now do ow that on the slight a for the s
in 39.654468059539795sec 25.0%) 1.8738]
Whing and vercace the _went of any
So it. O will plop it will, and pirst fort to fire
hen seap along shourded it. I fale of he cray down it, and up the 
in 59.58103942871094sec 30.0%) 1.7899]

Now you can generate more examples using a trained model. Recall that `generate` takes the mentioned below arguments to work:

```python
generate(decoder, prime_str='A', predict_len=100, temperature=0.8, device='cpu')
```

Try it by yourself

In [15]:
print(f"{generate(decoder, prime_str='Wh', predict_len=100, device=DEVICE)}\n")

Whermbles where that thind off and goor going there going of preceable didn't do niggered to far was t



---
# Section 2: Power consumption in Deep Learning

*Time estimate: ~20mins*

Training NN models can be incredibly costly, both in actual money but also in power consumption.

In [16]:
# @title Video 2: Carbon Footprint of AI
from ipywidgets import widgets

out2 = widgets.Output()
with out2:
  from IPython.display import IFrame
  class BiliVideo(IFrame):
    def __init__(self, id, page=1, width=400, height=300, **kwargs):
      self.id=id
      src = "https://player.bilibili.com/player.html?bvid={0}&page={1}".format(id, page)
      super(BiliVideo, self).__init__(src, width, height, **kwargs)

  video = BiliVideo(id=f"BV1My4y1j7HJ", width=854, height=480, fs=1)
  print("Video available at https://www.bilibili.com/video/{0}".format(video.id))
  display(video)

out1 = widgets.Output()
with out1:
  from IPython.display import YouTubeVideo
  video = YouTubeVideo(id=f"as6C334LmRs", width=854, height=480, fs=1, rel=0)
  print("Video available at https://youtube.com/watch?v=" + video.id)
  display(video)

out = widgets.Tab([out1, out2])
out.set_title(0, 'Youtube')
out.set_title(1, 'Bilibili')

# Add event to airtable
atform.add_event('Video 2: Carbon Footprint of AI')

display(out)

Take a few moments to chat with your pod about the following points:
* Which societal costs of training do you find most compelling?
* When is training an AI model worth the cost?  Who should make that decision?
* Should there be additional taxes on energy costs for compute centers? 

## Exercise 2: Calculate the carbon footprint that your pod generated today.

You can use this [online calculator](https://mlco2.github.io/impact/#compute). 

In [17]:
# @title Student Response
from ipywidgets import widgets


text=widgets.Textarea(
   value='Type your answer here and click on `Submit!`',
   placeholder='Type something',
   description='',
   disabled=False
)

button = widgets.Button(description="Submit!")

display(text,button)

def on_button_clicked(b):
   atform.add_answer('q1', text.value)
   print("Submission successful!")


button.on_click(on_button_clicked)

Textarea(value='Type your answer here and click on `Submit!`', placeholder='Type something')

Button(description='Submit!', style=ButtonStyle())

---
# Summary

What a day!  We've learned a lot!  The basics of CNNs and RNNs, and how changes to architecture that allow models to parameter share can greatly reduce the size of the model.  We learned about convolution and pooling, as well as the basic idea behind RNNs.  To wrap up we thought about the impact of training large NN models.

In [18]:
# @title Airtable Submission Link
from IPython import display as IPydisplay
IPydisplay.HTML(
   f"""
 <div>
   <a href= "{atform.url()}" target="_blank">
   <img src="https://github.com/NeuromatchAcademy/course-content-dl/blob/main/tutorials/static/SurveyButton.png?raw=1"
 alt="button link end of day Survey" style="width:410px"></a>
   </div>""" )